In [7]:
import os
import pandas as pd
import numpy as np
from glob import glob

# 1. Point here at your annotations CSVs
annotations_dir = "./annotations_csvs"
# 2. This is where we'll write the segment files
output_dir = "./segments_csvs"
os.makedirs(output_dir, exist_ok=True)

def segment_contiguous_labels(df, fps=25):
    labels = df["label"].values
    if len(labels)==0:
        return []
    segments = []
    current = labels[0]
    start = 0
    for i in range(1, len(labels)):
        if labels[i] != current:
            segments.append({
                "start_frame": start,
                "end_frame": i-1,
                "start_time": start/fps,
                "end_time": (i-1)/fps,
                "label": current
            })
            start, current = i, labels[i]
    segments.append({
        "start_frame": start,
        "end_frame": len(labels)-1,
        "start_time": start/fps,
        "end_time": (len(labels)-1)/fps,
        "label": current
    })
    return segments

# 3. Loop over each modality’s annotations CSV and write segments
for modality_csv in glob(os.path.join(annotations_dir, "annotations_*.csv")):
    df = pd.read_csv(modality_csv)
    all_segs = []
    for (vid, seg, pid), group in df.groupby(["video_id","segment","participant"]):
        group = group.sort_values("frame")
        segs = segment_contiguous_labels(group, fps=25)
        for s in segs:
            all_segs.append({
                "modality": os.path.basename(modality_csv).split("_")[1].replace(".csv",""),
                "video_id": vid,
                "segment": seg,
                "participant": pid,
                **s
            })
    out_name = os.path.basename(modality_csv).replace("annotations_","segments_")
    pd.DataFrame(all_segs).to_csv(os.path.join(output_dir, out_name), index=False)
    print(f"✅ Wrote {len(all_segs)} segments to {output_dir}/{out_name}")


✅ Wrote 14170 segments to ./segments_csvs/segments_no_audio.csv
✅ Wrote 10468 segments to ./segments_csvs/segments_with_audio.csv
✅ Wrote 12435 segments to ./segments_csvs/segments_only_audio.csv


In [8]:
import os
from glob import glob
import pandas as pd
from IPython.display import display

segments_dir = "./segments_csvs"
paths = glob(os.path.join(segments_dir, "*.csv"))

dfs = []
for path in paths:
    modality = os.path.basename(path).split("_")[1].replace(".csv","")
    df = pd.read_csv(path)
    df["modality"] = modality
    dfs.append(df)

all_segments = pd.concat(dfs, ignore_index=True)
all_segments["duration"] = all_segments["end_time"] - all_segments["start_time"]

summary = (
    all_segments
    .groupby(["modality","label"])["duration"]
    .agg(count="count", mean="mean", median="median", std="std")
    .reset_index()
)

print("Segment Duration Summary by Modality and Label:")
display(summary)


Segment Duration Summary by Modality and Label:


,modality,label,count,mean,median,std
0,no,0,7631,39.683533,15.32,62.981385
1,no,1,6539,3.050644,1.68,5.017325
2,only,0,6785,46.272690,16.60,68.477381
3,only,1,5650,1.980956,1.20,2.824189
4,with,0,5807,54.042790,19.60,76.719833
5,with,1,4661,2.482901,1.44,5.104247


In [11]:
# Step 1: Crear ventanas fijas etiquetadas por “Union Vote”
import os
import pandas as pd
import numpy as np

# Config
fps = 60  # frames per second
window_sizes = [1]  # en segundos
ann_csv = "./annotations_csvs/annotations_with_audio.csv"

# Carga anotaciones union-voted
df_ann = pd.read_csv(ann_csv)
df_ann = df_ann.sort_values(['video_id','segment','participant','frame'])

# Para cada tamaño de ventana, generamos un CSV con ventanas fijas
for w in window_sizes:
    frames_per_win = w * fps
    records = []

    # Agrupa por video/segment/participante
    for (vid, seg, pid), grp in df_ann.groupby(['video_id','segment','participant']):
        timeline = grp['label'].values
        n_wins   = int(np.ceil(len(timeline) / frames_per_win))

        for i in range(n_wins):
            start = int(i * frames_per_win)
            end   = int(start + frames_per_win)
            slice_lab = timeline[start:end]
            if len(slice_lab)==0:
                continue

            # Union Vote: si ANY frame == 1 → ventana positiva
            win_label = int(slice_lab.sum() > 0)
            records.append({
                'video_id': vid,
                'segment':  seg,
                'participant': pid,
                'window_i': i,
                'window_s': w,
                'label':    win_label
            })

    df_w = pd.DataFrame(records)
    out_path = f"./segments_csvs/segments_with_audio_{w}s.csv"
    df_w.to_csv(out_path, index=False)
    print(f"✅ Saved fixed-window annotations: {out_path} → {len(df_w)} windows")


✅ Saved fixed-window annotations: ./segments_csvs/segments_with_audio_1s.csv → 136450 windows
